In [1]:
import requests
import pandas as pd
import json
import re
import polars as pl
import Enum_data as ed
pl.Config.set_tbl_rows(1000)
from persiantools import characters, digits
import sqlite3

import general_functions as gf

In [ ]:
#---------------------------------------
symbol = "فخوز"
financial_year = 14031230
report_type = ed.names.Monthly_report.value
parse_date = gf.parse_date_persian(financial_year)
#----------------------------------------
sheet_num = ed.sheets[report_type].value
table = ed.tabels[report_type].value
reports = gf.get_results(symbol, parse_date, report_type, sheet_num)
#reports
date = list(reports.keys())[10]
period = int(reports[date]['period'])
publish = int(reports[date]['publish'])
url = reports[date]['url']

In [3]:
reports

{14030131: {'period': '1',
  'publish': 14030225,
  'title': 'گزارش فعالیت ماهانه دوره ۱ ماهه منتهی به  ۱۴۰۳/۰۱/۳۱(اصلاحیه)',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=P89KSyfsGz1p0AABRULQQQaQQQiw%3d%3d&rt=0&let=58&ct=0&ft=-1'},
 14030231: {'period': '1',
  'publish': 14030306,
  'title': 'گزارش فعالیت ماهانه دوره ۱ ماهه منتهی به  ۱۴۰۳/۰۲/۳۱',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=AUNwDejiF0q2MUO3mK60Uw%3d%3d&rt=0&let=58&ct=0&ft=-1'},
 14030331: {'period': '1',
  'publish': 14030404,
  'title': 'گزارش فعالیت ماهانه دوره ۱ ماهه منتهی به  ۱۴۰۳/۰۳/۳۱',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=WbytG37B1IftXDz7DxHPjQ%3d%3d&rt=0&let=58&ct=0&ft=-1'},
 14030431: {'period': '1',
  'publish': 14030503,
  'title': 'گزارش فعالیت ماهانه دوره ۱ ماهه منتهی به  ۱۴۰۳/۰۴/۳۱',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=7H7QQQaQQQLnyUlwPEtjBtKEd1Gg%3d%3d&rt=0&let=58&ct=0&ft=-1'},
 14030531: {'period': '1',
  'publish': 140

In [4]:
def get_table(url: str, table: int):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:143.0) Gecko/20100101 Firefox/143.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Origin': 'https://www.codal.ir',
    'Connection': 'keep-alive',
    'Referer': 'https://www.codal.ir/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'Cookie': 'TS018fb0f7=01f9930bd2e2675d04882f623c888052df60031e7775d8e3c459dc4cb96bf8e870e0a9bf0e0eedff68bcce23fd9701c7a7fc0855c4; Unknown=1076170924.20480.0000'
    }
    response = requests.request("GET", url)
    statement = response.text
    pattern = r"var datasource = (.*?});"
    match = re.search(pattern, statement)
    if match:
        text = match.group(1)
    records = []
    records.append(
        (statement, text))
    for _, data in records:
        continue
    items = json.loads(data)['sheets']

    if isinstance(table, list):
        cells = []
        for t in table:
            raw_cells = items[0]['tables'][t]['cells']
            cells.append([(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in raw_cells])
        return [x for xs in cells for x in xs]
    
    cells = items[0]['tables'][table]['cells']
    
    return [(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in cells]



In [ ]:
def create_dict_dataframes(url: str, date: int,  report_type: str) -> dict:
    st_date = f"{str(date)[:4]}/{str(date)[4:6]}/{str(date)[6:]}"
    all_data ={ 'report_this_month': pl.DataFrame(),
                'report_total': pl.DataFrame()}

    cells_tuples = get_table(url, ed.tabels[report_type].value)

    filtered_cells = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == '' or i[-1] == st_date]
    df = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
    df = df.pivot(values="value", on="col", index="row").sort("row")
    df = df.with_columns(pl.col('1').map_elements(characters.ar_to_fa, return_dtype=pl.String))
    
    df = df.drop(df.columns[3:6])
    common_cols = df.columns[:3]

    all_data['report_this_month'] = df.select(common_cols + df.columns[3:6])
    all_data['report_total'] = df.select(common_cols + df.columns[7:11])

    return all_data  
    

In [6]:
a = create_dict_dataframes(url, report_type)
a.keys()

dict_keys(['report_this_month', 'report_total'])

In [ ]:
def create_Monthly_report_dataframe(symbol: str, url: str, date: int, publish: int):
    all_data = create_dict_dataframes(url, 'Monthly_report')
    for name, df in all_data.items():
        df = df.rename({c: str(i) for i, c in enumerate(df.columns)})
        df = df.rename({'0': 'row'})
        table_date = [d for d in str(df[df.columns[3]][0]).split() if '/' in d][0]
        df_internal = df.clone()
        idx_internal = df_internal.filter(df_internal['1'] == "فروش داخلی:")['row'][0]
        idx_total_internal = df_internal.filter(df_internal['1'] == "جمع فروش داخلی")['row'][0]
        df_internal = df_internal.filter(df_internal['row'] > idx_internal, df_internal['row'] < idx_total_internal-1)[['1','2']]
        df_internal=df_internal.with_columns(pl.lit("داخلی").alias("3"))
        df_export=df.clone()
        idx_export = df_export.filter(df_export['1'] == "فروش صادراتی:")['row'][0]
        idx_total_export = df_export.filter(df_export['1'] == "جمع فروش صادراتی")['row'][0]
        df_export = df_export.filter(df_export['row']>idx_export ,df_export['row']<idx_total_export -1)[['1','2']]
        df_export = df_export.with_columns(pl.lit("صادراتی").alias('3'))
        df_products = pl.concat([df_internal,df_export])

        for i in df[0].rows(named=True):
            for k, v in i.items():
                if '/' in str(v):

                    col_num_production = k
                    dummy_data = df[col_num_production:str(int(col_num_production)+2)]
                    dummy_data = dummy_data.insert_column(0, df['row'])
                       
                    sales_internal = dummy_data.filter(dummy_data['row'] > idx_internal, dummy_data['row'] < idx_total_internal-1)
                    sales_export = dummy_data.filter(dummy_data['row']>idx_export ,dummy_data['row']<idx_total_export -1)
                    sales_products = pl.concat([sales_internal, sales_export])               
                    sales_products = sales_products.drop(sales_products.columns[0])
                    
                    #Rename columns to range(0,1,...)
                    sales_products = sales_products.rename(dict(zip(sales_products.columns, map(str, range(len(sales_products.columns))))))
                    #If "" is in the dataframe

                    if gf.has_empty_string(sales_products):
                        continue
                    sales_products = sales_products.with_columns(pl.col(pl.Utf8).replace("", "0").cast(pl.Int64)).filter(~pl.all_horizontal(pl.all() == 0))
                    #Convert string to Int from Col 1 to end
                    sales_products = sales_products.with_columns([pl.col(sales_products.columns).cast(pl.Int64)])
                    # concat label and values
                    df_products = df_products.rename({c: "_" + c for c in df_products.columns})
                    
                    data = pl.concat([df_products, sales_products], how="horizontal")

                    data.columns = [str(i) for i in range(data.width)]
                    data = data.insert_column(0, pl.lit(symbol).alias("Symbol"))
                    data = data.insert_column(1, pl.lit(int(digits.fa_to_en(table_date).replace('/', ''))).alias("Date"))
                    data = data.insert_column(2, pl.lit(publish).alias("Publish"))
                    data = data.drop_nulls()
                    all_data[name] = data
    return all_data

In [ ]:
create_Monthly_report_dataframe(symbol, url, date, publish)['report_total']

Symbol,Date,Publish,0,1,2,3,4,5
str,i32,i32,str,str,str,i64,i64,i64
"""فخوز""",14031130,14031205,"""اسلب""","""تن""","""داخلی""",882998,784967,237261091
"""فخوز""",14031130,14031205,"""گندله ""","""تن""","""داخلی""",4874922,15661,54727157
"""فخوز""",14031130,14031205,"""بلوم ، بیلت ""","""تن""","""داخلی""",1885077,1228379,230330361
"""فخوز""",14031130,14031205,"""آهن اسفنجی""","""تن""","""داخلی""",4071352,502107,125205002
"""فخوز""",14031130,14031205,"""بلوم ، بیلت ""","""تن""","""صادراتی""",0,566819,231345181
"""فخوز""",14031130,14031205,"""اسلب""","""تن""","""صادراتی""",0,103361,262062674
"""فخوز""",14031130,14031205,"""آهن اسفنجی""","""تن""","""صادراتی""",0,29390,132843688
